# Generador del Modelo CNN para Clasificador de Emociones

## Integrantes:
- Beltran Medina Carlos Daniel
- Beltran Ontiveros Karen Valeria

## Dataset

### 1. Importar las librerias necesarias

In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import numpy as np
from PIL import Image

### 2. Configuraciones previas

In [2]:
ruta_dataset = "../dataset"
batch_size = 64
epochs = 30
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


### 3. Carga y preprocesamiento de las imagenes

In [5]:
train_transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

val_transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(os.path.join(ruta_dataset, 'train'), transform=train_transforms)
val_dataset = datasets.ImageFolder(os.path.join(ruta_dataset, 'test'), transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print(f"Clases: {class_names}")

Clases: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


### 4. Modelo CNN

In [4]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(32), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(64), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.BatchNorm2d(128), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(256, 7)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc(x)
        return x
    
model = EmotionCNN().to(device)

### 5. Entrenamiento

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

best_val_acc = 0.0

for epoch in range(epochs):
    model.train()
    train_loss, train_correct = 0.0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += torch.sum(preds == labels).item()

    model.eval()
    val_loss, val_correct = 0.0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels).item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    train_acc = train_correct / len(train_dataset)
    val_acc = val_correct / len(val_dataset)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train loss: {train_loss/len(train_dataset):.4f}, acc: {train_acc:.4f}")
    print(f"Val   loss: {val_loss/len(val_dataset):.4f}, acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "modelo_emociones.pt")
        print("Modelo generado exitosamente.")

Epoch 1/30
Train loss: 1.3779, acc: 0.4725
Val   loss: 1.2554, acc: 0.5198
Modelo generado exitosamente.
Epoch 2/30
Train loss: 1.3034, acc: 0.5053
Val   loss: 1.1819, acc: 0.5525
Modelo generado exitosamente.
Epoch 3/30
Train loss: 1.2443, acc: 0.5273
Val   loss: 1.1604, acc: 0.5559
Modelo generado exitosamente.
Epoch 4/30
Train loss: 1.2125, acc: 0.5413
Val   loss: 1.1652, acc: 0.5606
Modelo generado exitosamente.
Epoch 5/30
Train loss: 1.1825, acc: 0.5532
Val   loss: 1.1212, acc: 0.5793
Modelo generado exitosamente.
Epoch 6/30
Train loss: 1.1560, acc: 0.5667
Val   loss: 1.1035, acc: 0.5886
Modelo generado exitosamente.
Epoch 7/30
Train loss: 1.1271, acc: 0.5720
Val   loss: 1.1012, acc: 0.5911
Modelo generado exitosamente.
Epoch 8/30
Train loss: 1.1108, acc: 0.5787
Val   loss: 1.0984, acc: 0.5883
Epoch 9/30
Train loss: 1.0907, acc: 0.5874
Val   loss: 1.0774, acc: 0.5995
Modelo generado exitosamente.
Epoch 10/30
Train loss: 1.0664, acc: 0.5986
Val   loss: 1.0815, acc: 0.5995
Epoch 11/

### 6. Reporte del modelo

In [9]:
print("Reporte de clasificación:")
print(classification_report(all_labels, all_preds, target_names=class_names))

Reporte de clasificación:
              precision    recall  f1-score   support

       angry       0.60      0.49      0.54       958
     disgust       0.80      0.37      0.51       111
        fear       0.47      0.43      0.45      1024
       happy       0.84      0.83      0.84      1774
     neutral       0.59      0.63      0.61      1233
         sad       0.47      0.56      0.51      1247
    surprise       0.77      0.77      0.77       831

    accuracy                           0.63      7178
   macro avg       0.65      0.58      0.60      7178
weighted avg       0.64      0.63      0.63      7178



## Prueba con la camara

### 1. Cargar el modelo

In [5]:
model = EmotionCNN().to(device)
model.load_state_dict(torch.load("modelo_emociones.pt", map_location=device))
model.eval()
class_names = ['Enojado', 'Disgustado', 'Aterrado', 'Feliz', 'Neutral', 'Triste', 'Asombrado']
print("Modelo cargado correctamente")

Modelo cargado correctamente


### 2. Encender la camara

In [ ]:
cap = cv2.VideoCapture(0)

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

if not cap.isOpened():
    print("No se pudo acceder a la cámara.")
    exit()

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

print("Cámara iniciada. Presiona 'q' para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (48, 48))
        face_pil = Image.fromarray(face_resized)
        face_tensor = transform(face_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(face_tensor)
            _, pred = torch.max(outputs, 1)
            emotion = class_names[pred.item()]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Detección de emociones', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Cámara iniciada. Presiona 'q' para salir.


: 